In [11]:
# install CUDA toolkit 11.6
!conda install -c nvidia -c conda-forge cudatoolkit=11.6 cuda-nvcc -y

done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
WARNING conda.models.version:get_matcher(542): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.9.0.*, but conda is ignoring the .* and treating it as 1.9.0
WARNING conda.models.version:get_matcher(542): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.6.0.*, but conda is ignoring the .* and treating it as 1.6.0
done
Solving environment: / 

In [10]:
# install Pytorch 11.6 and other deps
import sys
!{sys.executable} -m pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!{sys.executable} -m pip install timm Pillow submitit 

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/usr/bin/python3.9 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/usr/bin/python3.9 -m pip install --upgrade pip


In [12]:
# install APEX
import sys
!git clone https://github.com/NVIDIA/apex
!cd apex && git checkout 2386a912164b0c5cfcd8be7a2b890fbac5607c82
!{sys.executable} -m pip install -r apex/requirements.txt
!{sys.executable} -m pip install enscons cryptacular
!cd apex && {sys.executable} -m pip install -v --disable-pip-version-check --no-cache-dir --no-build-isolation --config-settings "--build-option=--cpp_ext" --config-settings "--build-option=--cuda_ext" ./

fatal: destination path 'apex' already exists and is not an empty directory.
HEAD is now at 2386a91 Distributed optimizer infrastructure for FP8 parameters (#1723)

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/usr/bin/python3.9 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/usr/bin/python3.9 -m pip install --upgrade pip
Using pip 23.2.1 from /media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/opt/python3.9/lib/python3.9/site-packages/pip (python 3.9)
Processing /u/h/gppl20/deit-softmax1/apex
  Running command Preparing metadata (pyproject.toml)


  torch.__version__  = 1.13.1+cu116


  running dist_info
  creating /tmp/pip-modern-metadata-x78jdu8k/apex.egg-info
  writing /tmp/pip-modern-metadata-x78jdu8k/apex.egg-info/PKG-INFO
  writing dependency_link

In [ ]:

# MOUNT & RE-ORGANIZE IMAGENET ZIP




import os
import shutil
import xml.etree.ElementTree as ET


# Kaggle imagenet1k path
zip = "/media/idmi/DISSERTATN/imagenet-object-localization-challenge.zip"
!fusermount -u /tmp/imgnet
!mkdir /tmp/imgnet
!fuse-zip {zip} /tmp/imgnet
#fusermount -u /tmp/imgnet










# Paths
val_image_dir = "/tmp/imgnet/ILSVRC/Data/CLS-LOC/val"
val_annotation_dir = "/tmp/imgnet/ILSVRC/Annotations/CLS-LOC/val"


# Function to parse XML and get class label
def get_class_label(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    object_element = root.find("object")
    name_element = object_element.find("name")
    print(name_element.text)
    return name_element.text


# Iterate through validation XML files
for xml_file in os.listdir(val_annotation_dir):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(val_annotation_dir, xml_file)
        class_label = get_class_label(xml_path)


        # Create the destination folder if it doesn't exist
        dest_folder = os.path.join(val_image_dir, class_label)
        os.makedirs(dest_folder, exist_ok=True)


        # Get the corresponding image file
        image_filename = xml_file.replace(".xml", ".JPEG")
        image_path = os.path.join(val_image_dir, image_filename)


        # Move the image to the correct class folder
        if os.path.exists(image_path):
            dest_path = os.path.join(dest_folder, image_filename)
            shutil.move(image_path, dest_path)


# Clean up: Remove empty folders in the val_image_dir
#for folder in os.listdir(val_image_dir):
#    folder_path = os.path.join(val_image_dir, folder)
#    if os.path.isdir(folder_path) and not os.listdir(folder_path):
#        os.rmdir(folder_path)

In [8]:
!{sys.executable} run_with_submitit.py --model deit_base_patch16_LS --data-path /media/gppl20/NTFS_portable/imagenet1k --output_dir /tmp/job --batch 256 --lr 3e-3 --epochs 800 --weight-decay 0.05 --sched cosine --input-size 192 --eval-crop-ratio 1.0 --reprob 0.0 --nodes 1 --ngpus 8 --smoothing 0.0 --warmup-epochs 5 --drop 0.0 --seed 0 --opt fusedlamb --warmup-lr 1e-6 --mixup .8 --drop-path 0.2 --cutmix 1.0 --unscale-lr --repeated-aug --bce-loss  --color-jitter 0.3 --ThreeAugment

^C
Traceback (most recent call last):
  File "/u/h/gppl20/deit-softmax1/run_with_submitit.py", line 11, in <module>
    import main as classification
  File "/u/h/gppl20/deit-softmax1/main.py", line 13, in <module>
    from timm.data import Mixup
  File "/media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/opt/python3.9/lib/python3.9/site-packages/timm/__init__.py", line 3, in <module>
    from .models import create_model, list_models, list_pretrained, is_model, list_modules, model_entrypoint, \
  File "/media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/opt/python3.9/lib/python3.9/site-packages/timm/models/__init__.py", line 1, in <module>
    from .beit import *
  File "/media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/opt/python3.9/lib/python3.9/site-packages/timm/models/beit.py", line 49, in <module>
    from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
  File "/media/gppl20/Debian_Portable/tmp/JUPYTER_P/python39.AppDir/opt/python3.9/lib/pyt

In [ ]:
!{sys.executable} run_with_submitit.py --softmax1_attn --model deit_base_patch16_LS --data-path /media/gppl20/NTFS_portable/imagenet1k --output_dir /tmp/job --batch 256 --lr 3e-3 --epochs 800 --weight-decay 0.05 --sched cosine --input-size 192 --eval-crop-ratio 1.0 --reprob 0.0 --nodes 1 --ngpus 8 --smoothing 0.0 --warmup-epochs 5 --drop 0.0 --seed 0 --opt fusedlamb --warmup-lr 1e-6 --mixup .8 --drop-path 0.2 --cutmix 1.0 --unscale-lr --repeated-aug --bce-loss  --color-jitter 0.3 --ThreeAugment